In [ ]:
from pynq import Overlay, MMIO
import time
import random

# Configuration
ARRAY_SIZE = 32
MAGIC_NUMBER = 0xDEADBEEF

# Memory layout
DATA_RAM_START = 0x2000 #Check this again
ARRAY_START_OFFSET = 0x40
STATUS_FLAG_OFFSET = 0x00

# Polling
MAX_TIMEOUT = 10  # seconds
POLL_DELAY = 0.001  # 1ms

In [ ]:
print("=" * 70)
print("Loading FPGA Bitstream...")
print("=" * 70)

ol = Overlay("design_1_wrapper.bit")

# Find GPIO and BRAM
gpio_key = next(k for k in ol.ip_dict if "gpio" in k.lower())
bram_key = next(k for k in ol.mem_dict if "axi_bram_ctrl" in k.lower() or "bram" in k.lower())

gpio_addr = int(ol.ip_dict[gpio_key]["phys_addr"])
gpio_range = int(ol.ip_dict[gpio_key]["addr_range"])
gpio = MMIO(gpio_addr, gpio_range)

mi = ol.mem_dict[bram_key]
bram_base = int(mi.get("phys_addr", mi.get("base_address")))
bram_size = int(mi.get("addr_range", mi.get("range")))
bram = MMIO(bram_base, bram_size)

print(f"GPIO: {gpio_key} @ {hex(gpio_addr)}")
print(f"BRAM: {bram_key} @ {hex(bram_base)} (size: {hex(bram_size)})")
print("Bitstream loaded successfully!")

In [ ]:
print("=" * 70)
print("Loading Instructions...")
print("=" * 70)

instructions = [
    "00000537",
    "04050513",
    "01f00293",
    "02028e63",
    "01f00313",
    "00050593",
    "02030463",
    "0005a383",
    "0045ae03",
    "007e2eb3",
    "000e8663",
    "01c5a023",
    "0075a223",
    "00458593",
    "fff30313",
    "fddff06f",
    "fff28293",
    "fc9ff06f",
    "0000006f"
]
# Write instructions to instruction ROM (starting at 0x0)
for i, instr_hex in enumerate(instructions):
    bram.write(i * 4, int(instr_hex, 16))

print(f"Loaded {len(instructions)} instructions to I-ROM")

In [ ]:
#Generating random array

print("=" * 70)
print("Generating Test Array...")
print("=" * 70)

# Generate random signed integers
random.seed(42)  # Remove this line for different array each time
test_array = [random.randint(-100, 100) for _ in range(ARRAY_SIZE)]

print(f"Generated {len(test_array)} random integers")
print(f"Range: -100 to 100")
print(f"\nTest array:")
print(test_array)

# Calculate expected result
golden_result = sorted(test_array)
print(f"\nExpected sorted result (first 10):")
print(golden_result[:10])

In [ ]:
#Inject test array into BRAM

print("=" * 70)
print("Injecting Test Array into BRAM...")
print("=" * 70)

# Reset status flag to 0
status_addr = DATA_RAM_START + STATUS_FLAG_OFFSET
bram.write(status_addr, 0x00000000)
print(f"Status flag reset @ {hex(status_addr)}")

# Write array to data RAM
array_base = DATA_RAM_START + ARRAY_START_OFFSET
for i, value in enumerate(test_array):
    # Convert signed to unsigned 32-bit
    unsigned_value = value & 0xFFFFFFFF
    bram.write(array_base + i * 4, unsigned_value)

print(f"Array injected @ {hex(array_base)}")

# Verify injection (read back first 8 values)
print("\nVerification (first 8 values):")
for i in range(8):
    read_val = bram.read(array_base + i * 4)
    signed_val = read_val if read_val < 0x80000000 else read_val - 0x100000000
    original = test_array[i]
    match = "✓" if signed_val == original else "✗"
    print(f"  [{i}] Expected: {original:4d}, Read: {signed_val:4d} {match}")

In [ ]:
#Start RISC-V Core (Release Reset)

print("=" * 70)
print("Starting RISC-V Core...")
print("=" * 70)

GPIO_DATA = 0x0
GPIO_TRI = 0x4

# Configure GPIO as output
gpio.write(GPIO_TRI, 0x0)

# Assert reset (low)
gpio.write(GPIO_DATA, 0x0)
print("Reset asserted (GPIO = 0)")
time.sleep(0.02)

# De-assert reset (high) - Core starts running!
gpio.write(GPIO_DATA, 0x1)
print("Reset de-asserted (GPIO = 1)")
print("RISC-V core is now running!")
time.sleep(0.02)

In [ ]:
#Poll for completion
print("=" * 70)
print("Waiting for Hardware to Complete Sorting...")
print("=" * 70)
print(f"Polling status flag @ {hex(status_addr)}")
print(f"Waiting for magic number: {hex(MAGIC_NUMBER)}")

start_time = time.time()
iterations = 0

while True:
    status = bram.read(status_addr)
    
    if status == MAGIC_NUMBER:
        elapsed = time.time() - start_time
        print(f"\n✓ COMPLETION DETECTED!")
        print(f"  Time: {elapsed:.3f} seconds")
        print(f"  Iterations: {iterations}")
        print(f"  Status flag: {hex(status)}")
        break
    
    if time.time() - start_time > MAX_TIMEOUT:
        elapsed = time.time() - start_time
        print(f"\n✗ TIMEOUT!")
        print(f"  Time: {elapsed:.3f} seconds")
        print(f"  Final status: {hex(status)}")
        print("  Hardware did not complete in time")
        break
    
    iterations += 1
    
    # Progress indicator
    if iterations % 1000 == 0:
        elapsed = time.time() - start_time
        print(f"  Polling... {iterations} iterations ({elapsed:.2f}s) - Status: {hex(status)}")
    
    time.sleep(POLL_DELAY)

In [ ]:
#Read Results from Hardware
print("=" * 70)
print("Reading Sorted Array from Hardware...")
print("=" * 70)

# Read sorted array from BRAM
result_array = []
for i in range(ARRAY_SIZE):
    read_val = bram.read(array_base + i * 4)
    # Convert unsigned back to signed
    signed_val = read_val if read_val < 0x80000000 else read_val - 0x100000000
    result_array.append(signed_val)

print(f"✓ Retrieved {len(result_array)} values from hardware")
print(f"\nHardware result (first 10):")
print(result_array[:10])
print(f"\nHardware result (last 10):")
print(result_array[-10:])

In [ ]:
#Verification of results
print("=" * 70)
print("VERIFICATION")
print("=" * 70)

print("\nComparison:")
print(f"Original array:  {test_array}")
print(f"Expected sorted: {golden_result}")
print(f"Hardware sorted: {result_array}")

# Check each element
mismatches = []
for i, (expected, actual) in enumerate(zip(golden_result, result_array)):
    if expected != actual:
        mismatches.append((i, expected, actual))

if len(mismatches) == 0:
    print("\n" + "=" * 70)
    print(" TEST PASSED! ")
    print("=" * 70)
    print(f"All {ARRAY_SIZE} values are correctly sorted!")
else:
    print("\n" + "=" * 70)
    print("✗✗✗ TEST FAILED ✗✗✗")
    print("=" * 70)
    print(f"Found {len(mismatches)} mismatches:")
    for idx, expected, actual in mismatches:
        print(f"  Index [{idx:2d}]: Expected {expected:4d}, Got {actual:4d}")

In [ ]:
#Optional Debug function
def dump_memory(start_addr, num_words, title="Memory Dump"):
    """Helper function to dump memory contents"""
    print(f"\n{title}")
    print("-" * 70)
    for i in range(num_words):
        addr = start_addr + i * 4
        value = bram.read(addr)
        signed = value if value < 0x80000000 else value - 0x100000000
        print(f"  [{hex(addr)}] word[{i:2d}] = {signed:6d} ({value:#010x})")

print("✓ Debug functions loaded")
print("  Use: dump_memory(address, num_words, title)")
print("  Example: dump_memory(array_base, 32, 'Array Contents')")

In [ ]:
#Quick retest function
def quick_retest(seed=None):
    """Quick function to re-run test with a new array"""
    print("=" * 70)
    print("QUICK RE-TEST")
    print("=" * 70)
    
    # Generate new array
    if seed is not None:
        random.seed(seed)
    new_array = [random.randint(-100, 100) for _ in range(ARRAY_SIZE)]
    print(f"New test array: {new_array}")
    
    # Inject
    bram.write(status_addr, 0x00000000)
    for i, value in enumerate(new_array):
        bram.write(array_base + i * 4, value & 0xFFFFFFFF)
    print("✓ Array injected")
    
    # Reset core
    gpio.write(GPIO_DATA, 0x0); time.sleep(0.02)
    gpio.write(GPIO_DATA, 0x1); time.sleep(0.02)
    print("✓ Core restarted")
    
    # Wait for completion
    start = time.time()
    while bram.read(status_addr) != MAGIC_NUMBER:
        if time.time() - start > MAX_TIMEOUT:
            print("✗ Timeout!")
            return
        time.sleep(POLL_DELAY)
    print(f"✓ Complete in {time.time() - start:.3f}s")
    
    # Read and verify
    hw_result = []
    for i in range(ARRAY_SIZE):
        val = bram.read(array_base + i * 4)
        hw_result.append(val if val < 0x80000000 else val - 0x100000000)
    
    expected = sorted(new_array)
    if hw_result == expected:
        print("✓✓✓ TEST PASSED!")
    else:
        print("✗✗✗ TEST FAILED!")
        for i, (e, a) in enumerate(zip(expected, hw_result)):
            if e != a:
                print(f"  Mismatch [{i}]: expected {e}, got {a}")

print("✓ Quick retest function loaded")
print("  Use: quick_retest()  or  quick_retest(seed=123)")

#quick_retest()